In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

result = load_dotenv(find_dotenv(), override=True)

if result:
    print("Variables de entorno cargadas exitosamente.")
else:
    print("No se pudo cargar el archivo .env.")

    # Crear el archivo .env con las variables deseadas si no existe
    with open(".env", "w") as f:
        f.write(f"OPENAI_API_KEY=MY_OPENAI_API_KEY\n")
        f.write(f"PINECONE_API_KEY=MY_PINECONE_API_KEY\n")
        f.write(f"PINECONE_CLUSTER=MY_PINECONE_CLUSTER\n")

    print("Se creó el archivo .env con las variables iniciales.")
    # Vuelve a cargar las variables de entorno después de crear el archivo
    load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"

# LLM Models (GPT-3.5)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

In [ ]:
gpt3 = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                  model_name='gpt-3.5-turbo-1106', 
                  temperature=0.7, max_tokens = 1024)

hacemos el modelo al que le vamos a pedir que nos responda, dandole la api_key, el nombre del modelo, la temperatura y el maximo de tokens

In [ ]:
print(gpt3)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Eres el mejor comendiante del mundo'),#contexto del modelo,
    ('user', '{input}')#mensaje
])

Utilizamos una plantilla de prompt donde en system le damos un contexto al modelo para que pueda responder, en user agregaremos el prompt

In [ ]:
chain = prompt | gpt3

Creamos la cadena uniendo el prompt y el modelo

In [ ]:
chain.invoke({"input":"dime un chiste corto"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser =  StrOutputParser()

In [ ]:
chain = prompt | gpt3 | output_parser

Para poder ver mejor el mensaje de respuesta del modelo, creamos una cadena agregandole la funcion de output

In [ ]:
with get_openai_callback() as cb:
    salida = chain.invoke({"input":"dime un chiste corto"})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

con la llamada a openai, aqui podemos ver la cantidad de tokens utolizados y el precio total de ellos

In [ ]:
print(salida)

# Cuenta de cuantos tokens puede tener una entrada y una salidad

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=0
)

texts = text_splitter.split_text(salida)

In [ ]:
print(texts)

In [ ]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

texts = text_splitter.split_text(salida)

In [ ]:
print(texts[0])